In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [3]:
query = """
WITH
user_df AS (SELECT *,
                  (CASE
                      WHEN nps_score <= 6 THEN "критики"
                      WHEN nps_score > 6 and nps_score <= 8 THEN "нейтралы"
                      WHEN nps_score > 8 THEN "cторонники"
                  END) AS nps_group,
                  (CASE
                      WHEN gender_segment = 0 THEN "мужской"
                      WHEN gender_segment = 1 THEN "женский"
                  END) AS gender
            FROM user),
location_df AS (SELECT *                      
                FROM location),
Lifetime_df AS (SELECT *,
                       (CASE
                            WHEN bucket_min <= 6 THEN "новый клиент"
                            WHEN bucket_min > 6 THEN "старый клиент"
                        END) AS is_new
                FROM lifetime_segment),
age_df AS (SELECT *
        FROM age_segment),
traffic_df AS (SELECT *
               FROM traffic_segment)
SELECT user_df.user_id,
       user_df.lt_day,
       user_df.age,
       user_df.os_name,
       user_df.cpe_type_name,
       user_df.nps_score,
       user_df.nps_group,
       user_df.gender,
       location_df.country,
       Location_df.city,
       lifetime_df.bucket_min AS lifetime_segment_min,
       Lifetime_df.bucket_max AS lifetime_segment_max,
       lifetime_df.is_new,
       age_df.bucket_min AS age_segment_min,
       age_df.bucket_max AS age_segment_max,
       age_df.title AS age_title,
       traffic_df.bucket_min AS traffic_segment_min,
       traffic_df.bucket_max AS traffic_segment_max,
       traffic_df.title AS traffic_title
FROM user_df
JOIN location_df ON user_df.location_id = location_df.location_id
JOIN Lifetime_df ON user_df.lt_gr_id = lifetime_df.lt_gr_id
JOIN age_df ON user_df.age_gr_id = age_df.age_gr_id
JOIN traffic_df ON user_df.tr_gr_id = traffic_df.tr_gr_id
"""

df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,age,os_name,cpe_type_name,nps_score,nps_group,gender,country,city,lifetime_segment_min,lifetime_segment_max,is_new,age_segment_min,age_segment_max,age_title,traffic_segment_min,traffic_segment_max,traffic_title
0,A001A2,2320,45.0,ANDROID,SMARTPHONE,10,cторонники,женский,Россия,Уфа,36.0,NaN,старый клиент,45.0,54.0,05 45-54,1.0,5.0,04 1-5
1,A001WF,2344,53.0,ANDROID,SMARTPHONE,10,cторонники,мужской,Россия,Киров,36.0,NaN,старый клиент,45.0,54.0,05 45-54,1.0,5.0,04 1-5
2,A003Q7,467,57.0,ANDROID,SMARTPHONE,10,cторонники,мужской,Россия,Москва,13.0,24.0,старый клиент,55.0,64.0,06 55-64,20.0,25.0,08 20-25


In [4]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

### Укажите ссылку на дашборд на сайте Tableau Public:

https://public.tableau.com/app/profile/barzord/viz/yandex_nps/Dashboard1?publish=yes

### Укажите ссылку на pdf-файл с презентацией:

https://docs.google.com/presentation/d/1mTk5XmMhOfjof7PO_hdqa4-RPvJ-r10O/edit?usp=sharing&ouid=101711347049068326300&rtpof=true&sd=true